In [5]:
# Import Packages
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler


# Dataset 1: Job Placement Dataset
Step 1
Question: Can we predict if a student will be placed in a job based on work experience, specialization, and academics?
Business Metric: Placement Rate

In [10]:
# 1. Load dataset
df_job_raw = pd.read_csv('Placement_Data_Full_Class.csv')

# 2. Step 2: Data Cleaning & Feature Engineering

# DROP LEAKAGE AND IDENTIFIERS FIRST
# We drop 'salary' here because it contains NaNs for all 'Not Placed' students.
# Dropping it first ensures dropna() doesn't delete the 'Not Placed' records.
df_job_reduced = df_job_raw.drop(columns=['salary', 'sl_no'], errors='ignore')

# Now remove rows with missing values in the REMAINING columns
df_job_cleaned = df_job_reduced.dropna().copy()

# 3. Define categorical columns
categorical_cols = [
    'gender', 'ssc_b', 'hsc_b', 'hsc_s',
    'degree_t', 'workex', 'specialisation', 'status'
]

# Convert types and simplify levels
df_job_cleaned[categorical_cols] = df_job_cleaned[categorical_cols].astype('category')
df_job_cleaned['specialisation'] = df_job_cleaned['specialisation'].apply(
    lambda x: x if x in ['Mkt&HR', 'Mkt&Fin'] else 'Other'
).astype('category')

# 4. One-hot encoding
df_job_encoded = pd.get_dummies(df_job_cleaned, columns=categorical_cols)

# 5. Normalize numeric variables
numeric_cols = list(df_job_encoded.select_dtypes('number'))
min_max_scaler = MinMaxScaler()
df_job_encoded[numeric_cols] = min_max_scaler.fit_transform(df_job_encoded[numeric_cols])

# 6. Create target variable (1 for Placed, 0 for Not Placed)
df_job_encoded['is_placed'] = df_job_encoded['status_Placed']

# Drop the redundant status columns now that we have our target
cols_to_drop = ["status_Placed", "status_Not Placed"]
df_job_final = df_job_encoded.drop(columns=[c for c in cols_to_drop if c in df_job_encoded.columns])

# 7. Calculate prevalence (Should now be ~68% instead of 100%)
placement_prevalence = df_job_final["is_placed"].mean()
print(f"Job Placement Prevalence: {placement_prevalence:.2%}")

# 8. Train / Tune / Test split
train_df, temp_df = train_test_split(
    df_job_final,
    train_size=0.7,
    stratify=df_job_final.is_placed,
    random_state=42
)

tune_df, test_df = train_test_split(
    temp_df,
    train_size=0.5,
    stratify=temp_df.is_placed,
    random_state=42
)

print("Final Split Shapes (Train, Tune, Test):")
print(train_df.shape, tune_df.shape, test_df.shape)

Job Placement Prevalence: 68.84%
Final Split Shapes (Train, Tune, Test):
(150, 22) (32, 22) (33, 22)


# Step 3: Concerns
Some concerns include potential bias from gender and academic background, salary dropped due to nulls and leakage risk, small dataset may cause overfitting, and placement depends on unobserved external factors

# Dataset 2: College Completion Dataset
Step 1
Question: Can we predict if a college has above-median graduation rates?
Business Metric: Graduation Rate

In [ ]:
# 1. Load dataset
df_college_raw = pd.read_csv("cc_institution_details.csv")

# 2. Target Creation & Initial Cleaning
# Create binary target: 1 if above median, 0 otherwise
grad_median_threshold = df_college_raw["grad_150_value"].median()
df_college_raw["is_high_grad"] = (df_college_raw["grad_150_value"] >= grad_median_threshold).astype(int)

# Identify columns to drop (Leakage, IDs, and Irrelevant metadata)
irrelevant_and_leakage_cols = [
    "index", "unitid", "chronname", "city", "site", "nicknames",
    "similar", "counted_pct", "long_x", "lat_y", "grad_150_value",
    "grad_100_value", "grad_100_percentile", "grad_150_percentile",
    "vsa_year", "vsa_grad_after4_first", "vsa_grad_elsewhere_after4_first",
    "vsa_enroll_after4_first", "vsa_enroll_elsewhere_after4_first",
    "vsa_grad_after6_first", "vsa_grad_elsewhere_after6_first",
    "vsa_enroll_after6_first", "vsa_enroll_elsewhere_after6_first",
    "vsa_grad_after4_transfer", "vsa_grad_elsewhere_after4_transfer",
    "vsa_enroll_after4_transfer", "vsa_enroll_elsewhere_after4_transfer",
    "vsa_grad_after6_transfer", "vsa_grad_elsewhere_after6_transfer",
    "vsa_enroll_after6_transfer", "vsa_enroll_elsewhere_after6_transfer"
]

# Create a cleaned version by dropping defined columns
df_college_cleaned = df_college_raw.drop(
    columns=[c for c in irrelevant_and_leakage_cols if c in df_college_raw.columns]
).copy()

# 3. Handle Missing Values (Imputation)
numeric_cols = df_college_cleaned.select_dtypes(include=[np.number]).columns
categorical_cols = df_college_cleaned.select_dtypes(include=["object"]).columns

for col in numeric_cols:
    if col != "is_high_grad":
        df_college_cleaned[col] = df_college_cleaned[col].fillna(df_college_cleaned[col].median())

for col in categorical_cols:
    df_college_cleaned[col] = df_college_cleaned[col].fillna(df_college_cleaned[col].mode()[0])

# 4. Feature Engineering: Collapse Carnegie Categories
if "basic" in df_college_cleaned.columns:
    def simplify_carnegie(category_name):
        category_name = str(category_name).lower()
        if "research" in category_name: return "Research"
        if "masters" in category_name: return "Masters"
        if "baccalaureate" in category_name: return "Baccalaureate"
        if "associate" in category_name: return "Associate"
        return "Other"

    df_college_cleaned["basic_category"] = df_college_cleaned["basic"].apply(simplify_carnegie)
    df_college_cleaned = df_college_cleaned.drop(columns=["basic"])

# Convert flags to integers
if "hbcu" in df_college_cleaned.columns:
    df_college_cleaned["is_hbcu"] = (df_college_cleaned["hbcu"] == "X").astype(int)
    df_college_cleaned = df_college_cleaned.drop(columns=["hbcu"])

if "flagship" in df_college_cleaned.columns:
    df_college_cleaned["is_flagship"] = (df_college_cleaned["flagship"] == "X").astype(int)
    df_college_cleaned = df_college_cleaned.drop(columns=["flagship"])

# 5. Encoding & Scaling
df_college_encoded = pd.get_dummies(df_college_cleaned, drop_first=True)

# Standardize numeric features
features_to_scale = [c for c in df_college_encoded.select_dtypes(include=[np.number]).columns if c != "is_high_grad"]
standard_scaler = StandardScaler()
df_college_encoded[features_to_scale] = standard_scaler.fit_transform(df_college_encoded[features_to_scale])

# 6. Calculate prevalence
college_grad_prevalence = df_college_encoded["is_high_grad"].mean()
print(f"College Graduation Prevalence (Baseline): {college_grad_prevalence:.2%}")

# 7. Train / Tune / Test split
train_college, temp_college = train_test_split(
    df_college_encoded,
    test_size=0.4,
    stratify=df_college_encoded["is_high_grad"],
    random_state=42
)

tune_college, test_college = train_test_split(
    temp_college,
    test_size=0.5,
    stratify=temp_college["is_high_grad"],
    random_state=42
)

print("Final College Dataset Shapes:")
print(f"Train: {train_college.shape}, Tune: {tune_college.shape}, Test: {test_college.shape}")

College Graduation Prevalence (Baseline): 45.68%
College dataset shapes:
Train: (2278, 86)
Tune: (760, 86)
Test: (760, 86)


/var/folders/cg/l51l5jms4ws7zn1vkks87spr0000gn/T/ipykernel_89076/36344224.py:36: Pandas4Warning: For backward compatibility, 'str' dtypes are included by select_dtypes when 'object' dtype is specified. This behavior is deprecated and will be removed in a future version. Explicitly pass 'str' to `include` to select them, or to `exclude` to remove them and silence this warning.
See https://pandas.pydata.org/docs/user_guide/migration-3-strings.html#string-migration-select-dtypes for details on how to write code that works with pandas 2 and 3.
  categorical_cols = college.select_dtypes(include=["object"]).columns


# Step 3: Concerns
There could Possible bias between public vs private institutions, socioeconomic-related variables may influence results, graduation rates affected by external events (e.g., pandemic), and median split may oversimplify institutional performance